In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil

In [33]:
main_dir = "/content/Brain_Tumor_Detection"

In [34]:
number_of_images = {}

In [35]:
for dir in os.listdir(main_dir):
  number_of_images[dir] = len(os.listdir(os.path.join(main_dir, dir)))

number_of_images

{'Brain_tumor': 1500, 'Healthy': 1500}

In [105]:
def datafolder(p, split):
  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

    for dir in os.listdir(main_dir):
      os.makedirs("./"+p+"/"+dir)
    for img in np.random.choice(a = os.listdir(os.path.join(main_dir, dir)), 
                                size = (math.floor(split*number_of_images[dir])-5),
                                replace = False):
      O = os.path.join(main_dir, dir, img)
      D = os.path.join("./"+p, dir)
      shutil.copy(O,D)
      os.remove(D)

  else:
    print(f"{p} Folder exist")

In [107]:
datafolder("Train", 0.7)

Train Folder exist


In [84]:
datafolder("Val", 0.15)

Val Folder exist


In [86]:
datafolder("Test", 0.15)

Test Folder exist


In [59]:
#model built

from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array
import keras
from keras.preprocessing.image import ImageDataGenerator

In [60]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation = 'relu', 
                 input_shape =(224,224,2)))
model.add(Conv2D(filters=32, kernel_size=(3,3)))

model.add(MaxPool2D(pool_size = (2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3)))
model.add(Conv2D(filters=128, kernel_size=(3,3)))

model.add(MaxPool2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(units=64, activation = 'relu'))
model.add(Dropout(rate = 0.25))
model.add(Dense(units=64, activation = 'sigmoid'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 222, 222, 16)      304       
                                                                 
 conv2d_18 (Conv2D)          (None, 220, 220, 32)      4640      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 110, 110, 32)     0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 108, 108, 64)      18496     
                                                                 
 conv2d_20 (Conv2D)          (None, 106, 106, 128)     73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 53, 53, 128)      0         
 2D)                                                  

In [61]:
model.compile(optimizer='adam', loss = keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [80]:
def preprocessingImage1(path):
  """
  Input : Path
  output : Pre Processed Path
  """
image_data = ImageDataGenerator(rescale = 1/255, zoom_range = 0.2, shear_range = 0.2,
                                horizontal_flip = True)
image = image_data.flow_from_directory(directory=r'/content/Train' ,
                                       target_size = (224,224), batch_size = 32,
                                       class_mode = 'binary')

Found 1 images belonging to 2 classes.


In [76]:
path = "/content/Train"

In [77]:
train_data = preprocessingImage1(path)

In [104]:
def preprocessingImage2(path):

    image_data = ImageDataGenerator(rescale = 1/255, zoom_range = 0.2, shear_range = 0.2,
                                horizontal_flip = True)
    image = image_data.flow_from_directory(directory=path ,
                                       target_size = (224,224), batch_size = 32,
                                       class_mode = 'binary')
    return image

In [90]:
path = "/content/Val"

In [92]:
Val_data = preprocessingImage2(path)

In [93]:
Test_data = preprocessingImage2(path)

In [101]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
es = EarlyStopping(monitor="Val_accuracy", min_delta=0.01,
                   patience=3, verbose=1, mode='auto')

mc = ModelCheckpoint(monitor="Val_accuracy", 
                     filepath="./Bestmodel.h5", verbose=1, 
                     save_best_only=True, mode='auto')

In [100]:
from keras import callbacks
hs= model.fit_generator(generator= train_data,
                        steps_per_epoch=8,
                        epochs=30,
                        verbose=1,
                        validation_data=Val_data,
                        validation_steps=16,
                        callbacks=mc)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


ValueError: ignored